In [2]:
from util.io.iter_BIDSPaths import *
from util.io.compute_stft import *

import gc
import sys
import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from typing import Tuple, Iterator
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree
from mne.time_frequency import tfr_morlet
from bids import BIDSLayout

from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from mne.decoding import SlidingEstimator, cross_val_multiscore

In [46]:
group = 'target'
fpath = '/project2/hcn1/pitch_tracking_attention/data/bids/derivatives/preprocessing/sub-27/sub-27_task-pitch_run-1_desc-clean_epo.fif.gz'
sub = '27'
run = '1'
STIM_FREQS = np.array([130, 200, 280])
FS = 5000
BIDS_ROOT = '../data/bids'
FIGS_ROOT = '../figs'

layout = BIDSLayout(BIDS_ROOT, derivatives = True)
fpaths = layout.get(scope = 'preprocessing',
            suffix='epo',
            extension = 'fif.gz',
            return_type = 'filename')

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [47]:
def summarize_stft(f, Zxx, n_epochs, condition_freqs): # approximate power at the condition freqs
    n_windows = np.shape(Zxx)[2]
    n_chans = np.shape(Zxx)[0]
    Zxx_condensed = np.empty([n_chans, len(condition_freqs), n_windows]) # n_chans, n_freqs, n_windows
    
    for i in range(len(condition_freqs)):
        
        # find indexes of freqs surrounding the condition freq
        freq = condition_freqs[i]
        a = int(np.argwhere(f < freq)[-1])
        b = a+1
        
        # subset power for the condition freq
        condition_Zxx = Zxx[:,a:b+1, :]
        
        # take average to get approximate for power at condition freq
        condition_Zxx = np.mean(condition_Zxx, axis = 1)
        Zxx_condensed[:, i, :] = condition_Zxx
    
    return Zxx_condensed

### Decode tone identity
Decode which tone is played for both target and non-target tones.

In [48]:
Zxxs = []
label = []
i = 0 
for (fpath, sub, task, run) in iter_BIDSPaths(fpaths):
    print(fpath)
    epochs = mne.read_epochs(fpath)
    events = epochs.events

    if group == 'target':
        CONDS = ['11', '22', '33']
        labels = [1, 2, 3]
    elif group == 'non_target':
        CONDS = ['12', '13', '21', '23', '31', '32']
        labels = [2, 3, 1, 3, 1, 2]

    for i, cond in enumerate(CONDS):
        print("---------- Subset epochs")
        condition_epochs = epochs[cond]
        events = condition_epochs.events
        print(condition_epochs.event_id)
        print(condition_epochs)
        print(np.shape(condition_epochs))
    
        print("---------- Take grand average in time domain")
        grand_avg = condition_epochs.average().get_data()
        print(np.shape(grand_avg))

        print("---------- Compute power")
        f, t, Zxx = signal.stft(grand_avg)
        print(np.shape(Zxx)) # array n_chans * n_freq * n_windows

        print("---------- Get power around frequencies of interest")
        Zxx_summarized = summarize_stft(f, Zxx, 1, STIM_FREQS)
        print(np.shape(Zxx_summarized))

        print("---------- Reshape data for decoder")
        n_chans = np.shape(Zxx_summarized)[0]
        n_freqs = np.shape(Zxx_summarized)[1]
        n_windows = np.shape(Zxx_summarized)[2]
        if n_chans != 62:
            print(f'INCORRECT NUMBER OF CHANNELS FOR SUB {sub} RUN {run}')
            continue
        Zxx_reshaped = Zxx_summarized.reshape((n_freqs*n_chans, n_windows)) # n_freqs*n_chans, n_windows
        print(np.shape(Zxx_reshaped))

        print("---------- Append to data frame")
        Zxxs.append(Zxx_reshaped) # n_subs (n_observations), n_freqs*n_chans, n_windows

        print("---------- Append condition label")
        label.append(labels[i])
        print(f'Labels: {label}')
    i += 1
    if i > 4:
        break


('/project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif.gz', '3', 'pitch', '1')
/project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif.gz
Reading /project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     350.00 ms
        0 CTF compensation matrices available
Reading /project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif-1.gz ...
    Found the data of interest:
        t =    -200.00 ...     350.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
0 bad epochs dropped
Not setting metadata
5193 matching events found
No baseline correction applied
0 projection items activated
-------

/tmp/ipykernel_49662/3960841544.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  Zxx_condensed[:, i, :] = condition_Zxx


{'22': 10005}
<EpochsFIF |  626 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~814.7 MB, data loaded,
 '22': 626>
(626, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
---------- Append condition label
Labels: [1, 2, 3]
---------- Subset epochs
{'33': 10009}
<EpochsFIF |  494 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~642.9 MB, data loaded,
 '33': 494>
(494, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
('/project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-4/sub-4_task-pitch_run-1_desc-clean_epo.fif.gz', '4', 'pitch', '1')
/

/tmp/ipykernel_49662/3960841544.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  Zxx_condensed[:, i, :] = condition_Zxx


{'33': 10009}
<EpochsFIF |  173 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~225.2 MB, data loaded,
 '33': 173>
(173, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
('/project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-5/sub-5_task-pitch_run-1_desc-clean_epo.fif.gz', '5', 'pitch', '1')
/project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-5/sub-5_task-pitch_run-1_desc-clean_epo.fif.gz
Reading /project2/hcn1/pitch_tracking_attention/analysis/../data/bids/derivatives/preprocessing/sub-5/sub-5_task-pitch_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     350.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
2079 ma

...

{'22': 10005}
<EpochsFIF |  369 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~480.3 MB, data loaded,
 '22': 369>
(369, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
---------- Append condition label
Labels: [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
---------- Subset epochs
{'33': 10009}
<EpochsFIF |  390 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~507.6 MB, data loaded,
 '33': 390>
(390, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decode

/tmp/ipykernel_49662/3960841544.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  Zxx_condensed[:, i, :] = condition_Zxx


{'22': 10005}
<EpochsFIF |  296 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~385.3 MB, data loaded,
 '22': 296>
(296, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
---------- Append condition label
Labels: [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
---------- Subset epochs
{'33': 10009}
<EpochsFIF |  225 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~292.9 MB, data loaded,
 '33': 225>
(225, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data f

/tmp/ipykernel_49662/3960841544.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  Zxx_condensed[:, i, :] = condition_Zxx


{'22': 10005}
<EpochsFIF |  275 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~357.9 MB, data loaded,
 '22': 275>
(275, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Reshape data for decoder
(186, 23)
---------- Append to data frame
---------- Append condition label
Labels: [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
---------- Subset epochs
{'33': 10009}
<EpochsFIF |  252 events (all good), -0.2 - 0.35 sec, baseline -0.2 – 0 sec, ~328.0 MB, data loaded,
 '33': 252>
(252, 62, 2751)
---------- Take grand average in time domain
(62, 2751)
---------- Compute power
(62, 129, 23)
---------- Get power around frequencies of interest
(62, 3, 23)
---------- Resha

In [51]:
print("---------- Create labels")
le = preprocessing.LabelEncoder()
y = le.fit_transform(label)

---------- Create labels


78

In [52]:
print("---------- Decode")
clf = make_pipeline(
    StandardScaler(),
    LogisticRegression(solver = 'liblinear')
)

print("Creating sliding estimators")
time_decod = SlidingEstimator(clf, scoring = 'balanced_accuracy')

print("Fit estimators")
scores = cross_val_multiscore(
    time_decod,
    Zxxs, # a trials x features x time array
    y, # an (n_observations) array of integer condition labels
    cv = 5, # use stratified 5-fold cross-validation
    n_jobs = -1, # use all available CPU cores
)
scores = np.mean(scores, axis = 0) # average across cv splits

---------- Decode
Creating sliding estimators
Fit estimators


ValueError: Found input variables with inconsistent numbers of samples: [69, 78]

In [ ]:
print("---------- Save decoder scores")
print('Saving scores to: ' + scores_fpath)
scores_fpath = '../data/bids/derivatives/decoding/' + cond + '_grand-avg.png'
np.save(scores_fpath, scores)

In [ ]:
print("---------- Plot")
fig, ax = plt.subplots()
ax.plot(range(len(scores)), scores, label = 'score')
ax.axhline(1/n_freqs, color = 'k', linestyle = '--', label = 'chance')
ax.set_xlabel('Times')
ax.set_ylabel('Accuracy')  # Area Under the Curve
ax.legend()
ax.set_title('Sensor space decoding')

# Save plot
fig_fpath = FIGS_ROOT + '/sub-' + sub + '_run-' + run + '_grand-avg' + '.png'
print('Saving figure to: ' + fig_fpath)
plt.savefig(fig_fpath)